In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [17]:
def calculate_F(X_star, X, y, alpha, b):
    result = []
    n_samples = X.shape[0]
    
    for i in range(n_samples):
        E = alpha[i] * y[i] * np.dot(X[i], X_star)
        result.append(E)
        
    f_x = np.sum(result) + b
    
    return f_x

In [18]:
def calculate_E(X_star, y_star, X, y, alpha, b):
    # calculate f(x)
    f_x = calculate_F(X_star, X, y, alpha, b)
    
    E_i = f_x - y_star
    
    return E_i

In [19]:
def generate_j(n_samples, i_idx):
    
    j = np.random.choice(n_samples)
    
    if j != i_idx:
        return j
    
    else:
        generate_j(n_samples, i_idx)

In [20]:
def calculate_boundaries(y_i, y_j, a_i, a_j, C):
    
    if y_i != y_j:
        L = max(0, a_j - a_i)
        H = min(C, C + a_j - a_i)
        
    else:
        L = max(0, a_i + a_j - C)
        H = min(C, a_i + a_j)
        
    return L, H

In [21]:
def calculate_eta(X_i, X_j):
    eta = 2 * (np.dot(X_i, X_j)) \
    - (np.dot(X_i, X_i)) - (np.dot(X_j, X_j))
    
    return eta

In [22]:
data_dict = {
    "x1": [0, 0, 1, 1],
    "x2": [0, 1, 0, 1],
    "y": [-1, 1, 1, 1]
}

In [23]:
data = pd.DataFrame(data = data_dict)

In [24]:
data

,x1,x2,y
0,0,0,-1
1,0,1,1
2,1,0,1
3,1,1,1


In [25]:
X = np.array(data[["x1", "x2"]])
X

array([[0, 0],
       [0, 1],
       [1, 0],
       [1, 1]])

In [26]:
y = np.array(data["y"])
y

array([-1,  1,  1,  1])

In [27]:
C = 0.1
tol = 1e-6
max_passes = 10

In [34]:
len(X)

4

In [35]:
X.shape[0]

4

In [28]:
n_samples = X.shape[0]
alpha = np.zeros(n_samples)
b = 0
passes = 0

In [31]:
while passes < max_passes:
    num_changed_alphas = 0
    
    for i in range(n_samples):
        # extract data index i
        
        X_i = X[i]
        y_i = y[i]
        a_i = alpha[i]
        
        # hitung E_i menggunakan persamaan (2)
        E_i = calculate_E(X_star = X_i,
                          y_star = y_i,
                          X = X,
                          y = y,
                          alpha = alpha,
                          b = b)
        
        cond_1 = ((y_i * E_i) < -tol) and (a_i < C)
        cond_2 = ((y_i * E_i) > tol) and (a_i > 0)
        
        if cond_1 or cond_2:
            # pilih j secara random
            j = generate_j(n_samples = n_samples,
                           i_idx = i)
            
            # extract data index j
            X_j = X[j]
            y_j = y[j]
            a_j = alpha[j]
            
            # hitung E_j menggunaan persamaan (2)
            E_j = calculate_E(X_star = X_j,
                              y_star = y_j,
                              X = X,
                              y = y,
                              alpha = alpha,
                              b = b)
            
            # update alpha values
            a_i_old = a_i
            a_j_old = a_j
            
            # hitung L dan H (upper and lower bound) menggunakan persamaan (10) dan (11)
            L, H = calculate_boundaries(y_i = y_i,
                                        y_j = y_j,
                                        a_i = a_i,
                                        a_j = a_j,
                                        C = C)
            
            # check boundaries condition
            if L == H:
                continue
                
            # calculate eta menggunakan persamaan (14)
            eta = calculate_eta(X_i = X_i,
                                X_j = X_j)
            
            # check eta condition
            if eta >= 0:
                continue
                
            # hitung dan clip new value dari a_j menggunakan (12) dan (15)
            a_j -= (y_j * (E_i - E_j)) / eta
            
            # condition to clip new value a_j
            if a_j > H:
                a_j = H
                
            elif L <= a_j and a_j <= H:
                a_j = a_j
                
            else:
                a_j = L
                
            # cek kondisi nilai a_j dengan tol
            if np.abs(a_j - a_j_old) < tol:
                continue
                
            # determine value a_i menggunakan persamaan (16)
            a_i += y_i * y_j * (a_j_old - a_j)
            
            # compute b1 dan b2 menggunakan persamaan (17) dan (18)
            b1 = b - E_i \
                 - y_i * (a_i - a_i_old) * np.dot(X_i, X_i) \
                 - y_j * (a_j - a_j_old) * np.dot(X_i, X_j)
            
            b2 = b - E_j \
                 - y_i * (a_i - a_i_old) * np.dot(X_i, X_j) \
                 - y_j * (a_j - a_j_old) * np.dot(X_j, X_j)
            
            # compute b menggunakan persamaan (19)
            if 0 < a_i and a_i < C:
                b = b1
                
            elif 0 < a_j and a_j < C:
                b = b2
                
            else:
                b = (b1 + b2) / 2
                
            alpha[i] = a_i
            alpha[j] = a_j
                
            num_changed_alphas += 1
    
    if num_changed_alphas == 0:
        passes += 1
        
    else:
        passes = 0

In [32]:
print(alpha)

[0.1  0.05 0.05 0.  ]


In [33]:
print(b)

0.95
